In [3]:
import numpy as np
import json
import os

import subprocess

In [4]:
%matplotlib inline
%config Completer.use_jedi = False

In [5]:
import subprocess

useless_cat_call = subprocess.run(["ls", "-lh"], stdout=subprocess.PIPE)
print(useless_cat_call.stdout)  # Hello from the other side

b'total 1936\ndrwxr-xr-x  138 sergmiller  staff   4.3K Nov  6 03:11 errorlogs\ndrwxr-xr-x  213 sergmiller  staff   6.7K Nov  6 03:11 replays\n-rw-r--r--    1 sergmiller  staff   965K Nov  6 13:02 research-v1.ipynb\n'


In [12]:
useless_cat_call.returncode

0

In [100]:
simple_bot = "../submissions/simple/main.py"
replays = "replays"

def run_game(left_bot=simple_bot, right_bot=simple_bot, seed=42, loglevel=2):
    replay_path = "replay.json"
    python_v = "python3.7"
    
    replay_path = os.path.join(replays, str(np.random.randint(1e9)) + ".json")
    
    size = np.random.choice([12,16,24,32], size=1)[0]
    
    res = subprocess.run([
        "lux-ai-2021",
        left_bot,
        right_bot,
#         "--statefulReplay",
        "--width={}".format(size),
        "--height={}".format(size),
        "--loglevel={}".format(loglevel),
        "--python={}".format(python_v),
        "--seed={}".format(seed),
        "--out={}".format(replay_path)], stdout=subprocess.PIPE)
    
    if loglevel > 0:
        print(res.stdout.decode())

    assert res.returncode == 0

    with open(replay_path, "r") as f:
        result = json.load(f)
    return result

In [48]:
import hashlib

def build_runnable_bot_with_flags(flags: dict, origin = simple_bot, base_path = '../submissions/simple/') -> str:
    lines = []
    with open(origin, "r") as f:
        for line in f:
            lines.append(line[:-1])
    text = '\n'.join(lines)
    f = json.dumps(flags)
    text = text.format(f)
    h = int(hashlib.sha256(f.encode('utf-8')).hexdigest(), 16) % (10 ** 9)
    path = base_path + "main_" + f + ".py"
    with open(path, "w") as f:
        f.write(text)
    return path

In [96]:
path = build_runnable_bot_with_flags({"debug": True})

In [97]:
path

'../submissions/simple/main_{"debug": true}.py'

In [101]:
r = run_game(simple_bot, simple_bot, 42)


-=-=-=-=-=-=-=-=-=-=-=-| [INFO] match_XQGm5T3Z5jkg |-=-=-=-=-=-=-=-=-=-=-=-

[INFO] (match_XQGm5T3Z5jkg) - Design: lux_ai_2021 | Initializing match - ID: XQGm5T3Z5jkg, Name: match_XQGm5T3Z5jkg
[WARN] (match_XQGm5T3Z5jkg) - turn 15; Unit u_2 collided when trying to move w to (23, 29)
[WARN] (match_XQGm5T3Z5jkg) - turn 15; Unit u_4 collided when trying to move s to (23, 29)
[WARN] (match_XQGm5T3Z5jkg) - turn 21; Unit u_6 collided when trying to move w to (22, 29)
[WARN] (match_XQGm5T3Z5jkg) - turn 25; Unit u_9 collided when trying to move e to (24, 27)
[WARN] (match_XQGm5T3Z5jkg) - turn 26; Unit u_10 collided when trying to move n to (21, 28)
[WARN] (match_XQGm5T3Z5jkg) - turn 28; Unit u_4 collided when trying to move n to (23, 26)
[WARN] (match_XQGm5T3Z5jkg) - turn 28; Unit u_13 collided when trying to move w to (23, 27)
[WARN] (match_XQGm5T3Z5jkg) - turn 29; Unit u_5 collided when trying to move n to (8, 27)
[WARN] (match_XQGm5T3Z5jkg) - turn 29; Unit u_7 collided when trying to move 

In [69]:
from joblib import Parallel, delayed

In [70]:
def run_series(l, r, seed=42, n_runs=10, n_jobs=4):
    import tqdm
    assert n_runs % 2 == 0
    results = []
#     for t in tqdm.tqdm(np.arange(n_runs), position=0):
#         if t % 2 == 0:
#             g = run_game(left_bot, right_bot, seed + t, loglevel=0)
#         else:
#             g = run_game(right_bot, left_bot, seed + t, loglevel=0)
#         results.append(g)
    
    with Parallel(n_jobs=n_jobs) as P:
        results = P(delayed(run_game)(l if t % 2 == 0 else r, r if t % 2 == 0 else l, seed + t, 0) for t in tqdm.tqdm(np.arange(n_runs), position=0))
    
    return results

In [29]:
def count_series(results: list):
    wins = []
    for i, r in enumerate(results):
        ranks = r['results']['ranks']
        teams = r['teamDetails']
        if ranks[0]['rank'] == 1 and ranks[1]['rank'] == 2:
            if ranks[0]["agentID"] == i % 2:
                wins.append(1)
            else:
                wins.append(0)
        else:
            wins.append(0.5)
    return wins

In [17]:
def calc_z_test(series: np.array):
    N = len(series)
    p = 0.5
    y = np.sum(series)
    z = (y - N * p) / (N * p * (1 - p)) ** 0.5
    
    from scipy.stats import norm
    def p_val(x):
        return 2 * min(norm.cdf(-x), norm.cdf(x))
    
    return y / N, z, p_val(z)

In [86]:
def build_bot_from_many(source) -> str:
    if isinstance(source, dict):
        s = build_runnable_bot_with_flags(source)
    else:
        assert isinstance(source, str)
        s = source
    return s

def compare_flags(flags_left, flags_right, n_games: int=10, seed: int=42):
    l = build_bot_from_many(flags_left)
    r = build_bot_from_many(flags_right)
    series = run_series(l, r, seed, n_games)
    counts = count_series(series)
    return calc_z_test(np.array(counts)), series[0]['results']

In [58]:
compare_flags({"alfa": 0.9}, {"alfa": 0.0}, n_games=20)

100%|██████████| 20/20 [00:53<00:00,  2.68s/it]


(0.475, -0.22360679774997896, 0.8230632737581215)

In [59]:
compare_flags({"alfa": 0.9}, {"alfa": 1.0}, n_games=20)

100%|██████████| 20/20 [00:56<00:00,  2.80s/it]


(0.5, 0.0, 1.0)

In [64]:
compare_flags({"alfa": 0.0}, {"alfa": 1.0}, n_games=20)

100%|██████████| 20/20 [00:59<00:00,  2.98s/it]


(0.3, -1.7888543819998317, 0.07363827012030266)

In [72]:
compare_flags({"alfa": 0.0}, {"alfa": 1.0}, n_games=40)

100%|██████████| 40/40 [00:40<00:00,  1.01s/it]


(0.35, -1.8973665961010275, 0.057779571123597245)

In [62]:
compare_flags({"norm_fuel_to_city_size": True}, {}, n_games=20)

100%|██████████| 20/20 [00:50<00:00,  2.51s/it]


(0.425, -0.6708203932499369, 0.5023349543605021)

In [76]:
compare_flags({"go_resource_next_action_probs": [0.25, 0.75]}, {}, n_games=40)

100%|██████████| 40/40 [00:42<00:00,  1.06s/it]


(0.675, 2.2135943621178655, 0.026856695507524397)

In [77]:
compare_flags({"go_resource_next_action_probs": [0.1, 0.9]}, {}, n_games=40)

100%|██████████| 40/40 [00:41<00:00,  1.05s/it]


(0.65, 1.8973665961010275, 0.057779571123597245)

In [83]:
compare_flags({"go_resource_next_action_probs": [0.25, 0.75], "norm_probs_to_city_tiles": True}, {}, n_games=40)

100%|██████████| 40/40 [00:50<00:00,  1.27s/it]


((0.575, 0.9486832980505138, 0.34278171114791145),
 {'ranks': [{'rank': 1, 'agentID': 1}, {'rank': 2, 'agentID': 0}],
  'replayFile': 'replays/34159252.json'})

In [85]:
zero_bot = "../submissions/zero_actions/main.py"
r = run_game(simple_bot, "/Users/sergmiller/Downloads/submission-5/main.py", 42)


-=-=-=-=-=-=-=-=-=-=-=-| [INFO] match_M2qKXLJ1wmsG |-=-=-=-=-=-=-=-=-=-=-=-

[INFO] (match_M2qKXLJ1wmsG) - Design: lux_ai_2021 | Initializing match - ID: M2qKXLJ1wmsG, Name: match_M2qKXLJ1wmsG
[WARN] (match_M2qKXLJ1wmsG) - turn 21; Unit u_4 collided when trying to move n to (2, 9)
[WARN] (match_M2qKXLJ1wmsG) - turn 26; Unit u_1 collided when trying to move n to (1, 9)
[WARN] (match_M2qKXLJ1wmsG) - turn 26; Unit u_6 collided when trying to move w to (1, 10)
[WARN] (match_M2qKXLJ1wmsG) - turn 27; Unit u_1 collided when trying to move w to (0, 10)
[WARN] (match_M2qKXLJ1wmsG) - turn 27; Unit u_6 collided when trying to move w to (1, 10)
[WARN] (match_M2qKXLJ1wmsG) - turn 27; Unit u_3 collided when trying to move n to (0, 10)
[WARN] (match_M2qKXLJ1wmsG) - turn 27; Unit u_7 collided when trying to move n to (0, 11)
[WARN] (match_M2qKXLJ1wmsG) - turn 35; Unit u_3 collided when trying to move n to (1, 10)
[WARN] (match_M2qKXLJ1wmsG) - turn 35; Unit u_6 collided when trying to move w to (1, 10

In [106]:
compare_flags({}, "/Users/sergmiller/Downloads/submission-5/main.py", n_games=40)

100%|██████████| 40/40 [01:02<00:00,  1.56s/it]


((0.4625, -0.4743416490252569, 0.6352562959972483),
 {'ranks': [{'rank': 1, 'agentID': 0}, {'rank': 2, 'agentID': 1}],
  'replayFile': 'replays/419589201.json'})

In [111]:
compare_flags({"check_only_adjacent_city_points": True}, {}, n_games=40)

100%|██████████| 40/40 [00:56<00:00,  1.40s/it]


((0.325, -2.2135943621178655, 0.026856695507524397),
 {'ranks': [{'rank': 1, 'agentID': 1}, {'rank': 2, 'agentID': 0}],
  'replayFile': 'replays/951213014.json'})

In [118]:
compare_flags({"skip_mine_and_build_loop": True}, {}, n_games=2)

100%|██████████| 2/2 [00:00<00:00, 12264.05it/s]


((0.0, -1.414213562373095, 0.15729920705028516),
 {'ranks': [{'rank': 1, 'agentID': 1}, {'rank': 2, 'agentID': 0}],
  'replayFile': 'replays/312348333.json'})

In [113]:
compare_flags({"go_to_city_at_night": True}, {}, n_games=40)

100%|██████████| 40/40 [01:11<00:00,  1.78s/it]


((0.475, -0.31622776601683794, 0.7518296340458492),
 {'ranks': [{'rank': 1, 'agentID': 0}, {'rank': 2, 'agentID': 1}],
  'replayFile': 'replays/41791872.json'})

In [117]:
compare_flags({"skip_go_to_city_on_zero_dist": True, "go_to_city_at_night": True}, {}, n_games=80)

100%|██████████| 80/80 [01:44<00:00,  1.30s/it]


((0.45625, -0.7826237921249264, 0.4338480657664392),
 {'ranks': [{'rank': 1, 'agentID': 0}, {'rank': 2, 'agentID': 1}],
  'replayFile': 'replays/657084024.json'})

In [116]:
compare_flags({"skip_go_to_city_on_zero_dist": True, "go_to_city_at_night": True}, "/Users/sergmiller/Downloads/submission-5/main.py", n_games=40)

100%|██████████| 40/40 [00:53<00:00,  1.34s/it]


((0.4375, -0.7905694150420948, 0.42919530044034926),
 {'ranks': [{'rank': 1, 'agentID': 1}, {'rank': 2, 'agentID': 0}],
  'replayFile': 'replays/295897956.json'})

In [125]:
compare_flags({"use_size_as_distance": True}, {}, n_games=40)

100%|██████████| 40/40 [00:42<00:00,  1.05s/it]


((0.3625, -1.7392527130926085, 0.081990321000383),
 {'ranks': [{'rank': 1, 'agentID': 1}, {'rank': 2, 'agentID': 0}],
  'replayFile': 'replays/39046897.json'})

In [123]:
r = run_game(build_runnable_bot_with_flags({"skip_mine_and_build_loop": True}), simple_bot)


-=-=-=-=-=-=-=-=-=-=-=-| [INFO] match_bet0zS3gBXZw |-=-=-=-=-=-=-=-=-=-=-=-

[INFO] (match_bet0zS3gBXZw) - Design: lux_ai_2021 | Initializing match - ID: bet0zS3gBXZw, Name: match_bet0zS3gBXZw
[Agent 0 Log] Traceback (most recent call last):
[Agent 0 Log]   File "main_{"skip_mine_and_build_loop": true}.py", line 42, in <module>
[Agent 0 Log]     actions = agent(observation, ENV)
[Agent 0 Log]   File "/Users/sergmiller/Documents/my/lux-ai-v1/submissions/simple/agent.py", line 208, in agent
[Agent 0 Log]     Routine.GO_BUILD_CITY: lambda: worker_try_to_build_a_city(unit)
[Agent 0 Log]   File "/Users/sergmiller/Documents/my/lux-ai-v1/submissions/simple/routines.py", line 40, in perform_routine
[Agent 0 Log]     state = update_state(unit, city_tiles, turn)
[Agent 0 Log]   File "/Users/sergmiller/Documents/my/lux-ai-v1/submissions/simple/routines.py", line 64, in update_state
[Agent 0 Log]     assert False
[Agent 0 Log] AssertionError
[WARN] (match_bet0zS3gBXZw) - turn 54; Unit u_3 collide